# 正则表达式

实现字符串的检索、替换、匹配

## 1.实例引入

开源中国提供的[正则表达式测试工具](https://tool.oschina.net/regex)

`[a-zA-z]+://[^\s]*`可以匹配URL

| 模式 | 描述 |
|:-:|:-:|
| `\w` | 字母、数字及下划线 |
| `\W` | 非\w |
| `\s` | 任意空白字符，等价于[\t\n\r\f] |
| `\S` | 非空字符 |
| `\d` | 数字 |
| `\D` | 非数字 |
| `\A` | 字符串开头 |
| `\Z` | 字符串结尾，换行前的字符 |
| `\z` | 字符串结尾，包括换行符 |
| `\G` | 最后匹配完成的位置 |
| `\n` | 换行符 |
| `\t` | 制表符 |
| `^` | 行开头 |
| `$` | 行结尾 |
| `.` | 任意字符，除了换行符 |
|  |  |
| `*` | 0个或多个前面表达式 |
| `+` | 1个或多个前面表达式 |
| `?` | 0个或1个前面表达式，非贪婪 |
| `{n}` | n个前面表达式 |
| `{n, m}` | n到m次前面表达式，贪婪 |
| `a|b` | a或b |
|  |  |


## 2.match

这里介绍第一个常用的匹配方法——match，向它传入要匹配的字符串以及正则表达式，就可以检测这个正则表达式是否和字符串相匹配。

In [4]:
import re

content = 'Hello 123 4567 World_This is a Regex Demo'
print(len(content))
result = re.match('^Hello\s\d\d\d\s\d{4}\s\w{10}', content)
print(result)
print(result.group())
print(result.span())

41
<re.Match object; span=(0, 25), match='Hello 123 4567 World_This'>
Hello 123 4567 World_This
(0, 25)


### 匹配目标

用match放法可以实现匹配，如果想从字符串中提取一部分，可以使用括号

In [16]:
import re

content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^Hello\s(\d+)\sWorld', content)
print(result)
print(result.group())
print(result.group(1))
print(result.span())

<re.Match object; span=(0, 19), match='Hello 1234567 World'>
Hello 1234567 World
1234567
(0, 19)


`group()`输出完整的匹配结果，`group(1)`输出第1个被()包括的匹配结果，以此类推。

### 通用匹配

刚才这样的工作量非常大。其实完全没必要这么做，因为还有一个万能匹配可以用，就是`.*`。`.`匹配任意字符（除换行符外），`*`匹配0次或多次，所以它们组合起来就可以匹配任意字符了。接上面例子，我们用`.*`改写一下上面例子。

In [17]:
import re

content = 'Hello 123 4567 World_This is a Regex Demo'
result = re.match('^Hello.*Demo$', content)
print(result)
print(result.group())
print(result.span())

<re.Match object; span=(0, 41), match='Hello 123 4567 World_This is a Regex Demo'>
Hello 123 4567 World_This is a Regex Demo
(0, 41)


### 贪婪与非贪婪

使用通用匹配`.*`匹配到的内容有时候不是我们想要的结果，请看下面例子：

In [19]:
import re

content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^He.*(\d+).*Demo$', content)
print(result)
print(result.group(1))

<re.Match object; span=(0, 40), match='Hello 1234567 World_This is a Regex Demo'>
7


为什么只得到了7这个数字？这里涉及到贪婪匹配和非贪婪匹配的问题。在贪婪匹配下，`.*`会匹配尽可能多的字符。正则表达式中的`.*`后面是`\d+`，也就是至少一个数字，而且没指定具体几个数字，因此，`.*`会匹配尽可能多的字符，这里就把`123456`都匹配了，只给`\d+`留下一个可满足条件的数字7，因此最后所得到的内容就只有数字7。

其实这里只需要使用非贪婪匹配就好了，非贪婪匹配的写法是`.*?`，比通用匹配多了一个`?`。

In [2]:
import re

content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^He.*?(\d+).*Demo$', content)
print(type(result))
print(result)
print(result.group(1))

<class 're.Match'>
<re.Match object; span=(0, 40), match='Hello 1234567 World_This is a Regex Demo'>
1234567


贪婪匹配是匹配尽可能多的字符，非贪婪匹配是匹配尽可能少的字符。

所以说，在做匹配的时候，字符串中间尽量使用非贪婪匹配，以免出现匹配结果缺失的情况；如果匹配结果在字符串结尾，`.*?`可能匹配不到任何内容了，因为它会匹配尽可能少的字符。例如：

In [6]:
import re

content = 'http://weibo.com/comment/kEraCN'
result1 = re.match('http.*?comment/(.*?)', content)
result2 = re.match('http.*?comment/(.*)', content)
print('result1:', result1.group(1))
print('result2:', result2.group(1))

result1: 
result2: kEraCN


可以观察到，`.*?`没有匹配到任何结果，而`.*`则是尽可能多匹配内容，成功得到了匹配结果。我们这里分析一下，到`http://weibo.com/comment/`是括号前的匹配内容，而`.*?`匹配尽可能少，因此可以一个都不匹配，即直接断掉，无内容，另一种方法可以在结尾加上`$`行字符串结尾符号，这样`kEraCN`就不得不被`(.*?)`所匹配了。

In [7]:
import re

content = 'http://weibo.com/comment/kEraCN'
result1 = re.match('http.*?comment/(.*?)', content)
result2 = re.match('http.*?comment/(.*?)$', content)
print('result1:', result1.group(1))
print('result2:', result2.group(1))

result1: 
result2: kEraCN


### 修饰符

在正则表达式中，可以用一些可选标志修饰符来控制匹配的模式。

In [12]:
import re

content = '''Hello 1234567 World_This
is a Regex Demo
'''
result = re.match('^He.*?(\d+).*?Demo$', content)
print(result.group(1))

AttributeError: 'NoneType' object has no attribute 'group'

发现运行直接报错，也就是说正则表达式没有匹配到这个字符串，返回结果为`None`，我们又调用了`group`方法，导致`AttributeError`。

那么，为什么加了一个换行符，就匹配不到了呢？因为`.`匹配的是除了换行符的任意字符，所以导致匹配失败。这里只需要加一个修饰符`re.S`，即可修正这个错误。这个修饰符的作用是使匹配内容包括换行符在内的所有字符。

这个`re.S`在网页匹配中经常用到，因为HTML节点经常会有换行，加上它，就可以匹配节点与节点之间的换行了

In [13]:
import re

content = '''Hello 1234567 World_This
is a Regex Demo
'''
result = re.match('^He.*?(\d+).*?Demo$', content, re.S)
print(result.group(1))

1234567


还有一些修饰符：

| 修饰符 | 描述 |
|---|---|
| `re.I` | 大小写不敏感 |
| `re.L` | 本地化识别（locale-aware）匹配 |
| `re.M` | 多行匹配，影响`^`和`$` |
| `re.S` | 使匹配内容包括换行符在内的所有字符 |
| `re.U` | 根据Unicode字符集解析字符，会影响`\w`、`\W`、`\b`和`\B` |
| `re.X` | 给予更灵活的格式，更易于理解 |

在网页匹配中，较为常用的有包括换行符`re.S`和大小写不敏感`re.I`

### 转义匹配

我们知道正则表达式定义了许多匹配模式，如`.`用于匹配除换行符以外的任意字符。但如果目标字符中有`.`，这时候就需要用到转义匹配了。

当在目标字符串中遇到用作正则匹配模式的特殊字符时，在此字符前面加`\`转义一下即可。

In [20]:
import re

content = '(百度)www.baidu.com'
result = re.match('(百度)www.baidu.com', content)
print(result)

None


In [21]:
import re

content = '(百度)www.baidu.com'
result = re.match('\(百度\)www\.baidu\.com', content)
print(result)

<re.Match object; span=(0, 17), match='(百度)www.baidu.com'>


## 3.search

`match`方法是从字符串的开头开始匹配的，意味着一旦开头不匹配，整个匹配就失败了。因为`match`方法在使用时需要考虑目标字符串开头的内容，因此在做匹配的时候并不方便。`match`更适合检测某个字符串是否符合某个正则表达式的规则。

In [27]:
import re

content = 'Extra stings Hello 1234567 World_This is a Regex Demo Extra      stings'
result = re.match('Hello.*?(\d+).*?Demo', content)
print(result)

None


这里有另一个方法`search`，它在匹配时会扫描整个字符串，然后返回第一个匹配成功的结果。因此，为了匹配方便，尽量使用`search`方法

In [1]:
import re

content = 'Extra stings Hello 1234567 World_This is a Regex Demo Extra      stings'
result = re.search('Hello.*?(\d+).*?Demo', content)
print(result)
print(result.group(1))

<re.Match object; span=(13, 53), match='Hello 1234567 World_This is a Regex Demo'>
1234567


尝试提取出`class`为`active`的li节点内部的歌手名与歌名

In [5]:
import re

html = '''<div id="songs-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路上有你</li>
<li data-view="7">
<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
</li>
<li data-view="4" class="active">
<a href="/3.mp3" singer="齐秦">往事随风</a>
</li>
<li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
<li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
<li data-view="5">
<a href="/6.mp3" singer="邓丽君">但愿人长久</a>
</li>
</ul>
</div>'''
result = re.search('<li.*?active.*?singer="(.*?)">(.*?)</a>', html, re.S)
print(result.group(1), result.group(2))

齐秦 往事随风


如果正则表达式不加`active`，会怎么样呢？

In [12]:
import re

html = '''<div id="songs-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路上有你</li>
<li data-view="7">
<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
</li>
<li data-view="4" class="active">
<a href="/3.mp3" singer="齐秦">往事随风</a>
</li>
<li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
<li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
<li data-view="5">
<a href="/6.mp3" singer="邓丽君">但愿人长久</a>
</li>
</ul>
</div>'''
result = re.search('<li.*?singer="(.*?)">(.*?)</a>', html, re.S)
print(result.group(1), result.group(2))

任贤齐 沧海一声笑


去掉`active`标签后，只会匹配第一个符合项。
如果把换行匹配去掉，会怎么样？

In [1]:
import re

html = '''<div id="songs-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路上有你</li>
<li data-view="7">
<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
</li>
<li data-view="4" class="active">
<a href="/3.mp3" singer="齐秦">往事随风</a>
</li>
<li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
<li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
<li data-view="5">
<a href="/6.mp3" singer="邓丽君">但愿人长久</a>
</li>
</ul>
</div>'''
result = re.search('<li.*?singer="(.*?)">(.*?)</a>', html)
print(type(result))
print(result.group(1), result.group(2))

<class 're.Match'>
beyond 光辉岁月


由于绝大部份HTML文本包含换行符，所以尽量加上`re.S`修饰符，以免出现匹配不到的问题。

## 4.findall

`search`方法可以返回匹配的第一个字符串，如果需要获取所有匹配结果，就需要用到`findall`

In [18]:
import re

html = '''<div id="songs-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路上有你</li>
<li data-view="7">
<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
</li>
<li data-view="4" class="active">
<a href="/3.mp3" singer="齐秦">往事随风</a>
</li>
<li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
<li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
<li data-view="5">
<a href="/6.mp3" singer="邓丽君">但愿人长久</a>
</li>
</ul>
</div>'''
results = re.findall('<li.*?href="/(\d+)\.mp3"\ssinger="(.*?)">(.*?)</a>', html, re.S)
# print(results)
print(type(results))
print(type(results[0]))
for each in results:
    # print(each[0], each[1], each[2])
    print(each)

<class 'list'>
<class 'tuple'>
('2', '任贤齐', '沧海一声笑')
('3', '齐秦', '往事随风')
('4', 'beyond', '光辉岁月')
('5', '陈慧琳', '记事本')
('6', '邓丽君', '但愿人长久')


In [47]:
import re

string = 'abc123efg456hij'

result1 = re.match('\w*?(\d+)\w*?(\d+)', string)
print(type(result1))
print(result1[1], result1[2])
print(result1.group(1), result1.group(2))

result2 = re.search('\w*?(\d+)\w*?(\d+)', string)
print(type(result2))
print(result2[1], result2[2])
print(result2.group(1), result2.group(2))

result3 = re.findall('\w*?(\d+)\w*?(\d+)', string)
# print(result3.group(1), result3.group(2))     # 无此方法
print(type(result3))
print(result3)
print(type(result3[0]))
print(result3[0])
print(result3[0][0], result3[0][1])

<class 're.Match'>
123 456
123 456
<class 're.Match'>
123 456
123 456
<class 'list'>
[('123', '456')]
<class 'tuple'>
('123', '456')
123 456


可以看到，返回的列表中的每个元素都是元组。
总结一下，如果只想获取匹配到的第一个字符串，可以用`search`；如果需要提取多个内容，可以用`findall`方法。
另外值得注意的一点是：`match`、`search`方法返回的是`Match`类，可以通过`group`或者类似数组下标的方式获取到；而`findall`方法返回的是元祖列表，只可以通过下标的方式访问到，没有`group`方法。

## 5.sub

除了使用正则表达式提取信息，有时候还需要借助它来修改文本。例如把遗传文本中的所有数字都去掉，如果只用字符串的`replace`方法，未免太繁琐了，这是就可以借助`sub`方法。

In [1]:
import re

content = '12a23bc78d676e7f9gh'
content = re.sub('\d+', '', content)
print(content)

abcdefgh


在上面的HTML文本中，如果想获取所有`li`节点的歌曲，直接用正则表达式来提取可能比较麻烦

In [63]:
import re

html = '''<div id="songs-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路上有你</li>
<li data-view="7">
<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
</li>
<li data-view="4" class="active">
<a href="/3.mp3" singer="齐秦">往事随风</a>
</li>
<li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
<li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
<li data-view="5">
<a href="/6.mp3" singer="邓丽君">但愿人长久</a>
</li>
</ul>
</div>'''
# results = re.findall('<li.*?>(.*?)</li>', html, re.S)
# results = re.findall('<li.*?>\s*?(<a.*?>)*(\w+)(</a>)*\s*?</li>', html, re.S)
results = re.findall('<li.*?>\s*?(<a.*?>)?(\w+)(</a>)?\s*?</li>', html, re.S)
for result in results:
    print(result[1])

一路上有你
沧海一声笑
往事随风
光辉岁月
记事本
但愿人长久


可以见到，匹配的过程非常地繁琐，而且非常容易出错。而此时借助`sub`方法就比较简单了。可以先用sub方法将a节点去掉，只留下文本，然后再利用`findall`提取就好了：

In [72]:
import re

html = '''<div id="songs-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路上有你</li>
<li data-view="7">
<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
</li>
<li data-view="4" class="active">
<a href="/3.mp3" singer="齐秦">往事随风</a>
</li>
<li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
<li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
<li data-view="5">
<a href="/6.mp3" singer="邓丽君">但愿人长久</a>
</li>
</ul>
</div>'''

html = re.sub('<a.*?>|</a>', '', html)
print(html)
results = re.findall('<li.*?>(.*?)</li>', html, re.S)
print(results)
for result in results:
    print(result.strip())

<div id="songs-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路上有你</li>
<li data-view="7">
沧海一声笑
</li>
<li data-view="4" class="active">
往事随风
</li>
<li data-view="6">光辉岁月</li>
<li data-view="5">记事本</li>
<li data-view="5">
但愿人长久
</li>
</ul>
</div>
['一路上有你', '\n沧海一声笑\n', '\n往事随风\n', '光辉岁月', '记事本', '\n但愿人长久\n']
一路上有你
沧海一声笑
往事随风
光辉岁月
记事本
但愿人长久


`strip`方法用于移除字符串头尾指定的字符（默认为空格或换行符）。

可以看到，经过`sub`方法处理后，a节点就没有了，然后通过`findall`方法直接提取即可。可以发现，在适当的时候借助`sub`方法，可以起到事半功倍的效果。

## 6.compile

前面所讲的方法都是用来处理字符串的方法，最后再介绍一下`compile`方法，这个方法可以将正则字符串编译成正则表达式对象，以便在以后的匹配中复用。

In [73]:
import re

content1 = '2019-12-15 12:00'
content2 = '2019-12-17 12:55'
content3 = '2019-12-22 13:21'

pattern = re.compile('\d{2}:\d{2}')
result1 = re.sub(pattern, '', content1)
result2 = re.sub(pattern, '', content2)
result3 = re.sub(pattern, '', content3)
print(result1, result2, result3)

2019-12-15  2019-12-17  2019-12-22 


`{2}`的意思是匹配两次前面表达式，就是时间，用`sub`方法便可去掉时间。

这里没必要重复写3个同样的正则表达式，此时就可以借助`compile`方法将正则表达式编译成一个正则表达式对象，以便复用

另外，`compile`方法还可以传入修饰符，例如`re.S`等，这样在`search`、`findall`等方法中就不需要额外传了。所以，可以说`compile`方法是给正则表达式做了一层封装，以便我们更好地复用。